In [ ]:
import numpy as np
from scipy.interpolate import interp1d

HAND_DICT = {
    'P01': 'right',
    'P02': 'right',
    'P03': 'right',
    'P04': 'right',
    'P05': 'left',
    'P06': 'right',
    'P07': 'right',
    'P08': 'right',
    'P09': 'right',
    'P10': 'left',
}

SMPL_JOINT_ORDER = [
    "Root",
    "Left Hip",
    "Right Hip",
    "Spine1",
    "Left Knee",
    "Right Knee",
    "Spine2",
    "Left Ankle",
    "Right Ankle",
    "Spine3",
    "Left Foot",
    "Right Foot",
    "Neck",
    "Left Collar",
    "Right Collar",
    "Head",
    "Left Shoulder",
    "Right Shoulder",
    "Left Elbow",
    "Right Elbow",
    "Left Wrist",
    "Right Wrist",
    "Prop"
]

def compute_ade_fde_jointsets(data1, data2, joint_indices):
    if data1.shape != data2.shape:
        raise ValueError("The shape of prediction and ground truth must be the same")
    
    data1 = data1[:, joint_indices, :]
    data2 = data2[:, joint_indices, :]
    
    distances = np.linalg.norm(data1 - data2, axis=(1, 2))
    ade = np.mean(distances)
    fde = distances[-1]
    
    return ade, fde
        
def interpolate_motion_data(data, tar_nframes):

    org_nframes, joints, coordinates = data.shape

    original_time = np.linspace(0, 1, org_nframes)
    target_time = np.linspace(0, 1, tar_nframes)
    
    interpolated_data = np.zeros((tar_nframes, joints, coordinates))
    
    for j in range(joints):
        for c in range(coordinates):
            interp_func = interp1d(original_time, data[:, j, c], kind='linear')
            interpolated_data[:, j, c] = interp_func(target_time)
    
    return interpolated_data


def crop_motion_data(data, tar_nframes):

    return data[:tar_nframes]


In [ ]:
import json
pred_dir = 'Submission_anonymized.json'
with open(pred_dir, 'r') as f:
    pred_data = json.load(f)
pred_dict = {item['seq_name']: item['motion'] for item in pred_data}


gt_dir = 'GT_anonymized.json'
with open(gt_dir, 'r') as f:
    gt_data = json.load(f)
gt_dict = {item['seq_name']: item['motion'] for item in gt_data}


All Joints

In [ ]:
import os
import numpy as np
import json

ade = 0
fde = 0
count = 0
for seq_name in pred_dict:
    
    pred_flat = pred_dict[seq_name] # nFrame x 23 x 3
    gt_flat = gt_dict[seq_name] # nFrame x 23 x 3

    T_pred = len(pred_flat) // (len(SMPL_JOINT_ORDER) * 3)
    T_gt = len(gt_flat) // (len(SMPL_JOINT_ORDER) * 3)

    pred = np.array(pred_flat).reshape((T_pred, len(SMPL_JOINT_ORDER), 3))
    gt = np.array(gt_flat).reshape((T_gt, len(SMPL_JOINT_ORDER), 3))
    
    # normalize 
    pred = pred - pred[:,:1,:]
    pred[:, :, 1], pred[:, :, 2] = pred[:, :, 2].copy(), pred[ :, :, 1].copy()
    gt = gt - gt[:,:1,:]
    
    if pred.shape !=gt.shape:
        pred = interpolate_motion_data(pred, gt.shape[0])

    selected_joints_idx = [SMPL_JOINT_ORDER.index(k) for k in SMPL_JOINT_ORDER]
    c_ade, c_fde = compute_ade_fde_jointsets(pred, gt, selected_joints_idx)
        
    ade += c_ade 
    fde += c_fde 
    count += 1
    
ade = ade/count
fde = fde/count
    
print('ADE (All Joints): ', ade)
print('FDE (All Joints): ', fde)


Limb(Arm)

In [ ]:
import os
import numpy as np
import json

ade = 0
fde = 0
count = 0
for seq_name in pred_dict:
    
    pred_flat = pred_dict[seq_name] # nFrame x 23 x 3
    gt_flat = gt_dict[seq_name] # nFrame x 23 x 3

    T_pred = len(pred_flat) // (len(SMPL_JOINT_ORDER) * 3)
    T_gt = len(gt_flat) // (len(SMPL_JOINT_ORDER) * 3)

    pred = np.array(pred_flat).reshape((T_pred, len(SMPL_JOINT_ORDER), 3))
    gt = np.array(gt_flat).reshape((T_gt, len(SMPL_JOINT_ORDER), 3))
    
    # normalize 
    pred = pred - pred[:,:1,:]
    pred[:, :, 1], pred[:, :, 2] = pred[:, :, 2].copy(), pred[ :, :, 1].copy()
    gt = gt - gt[:,:1,:]
    
    username = seq_name.split('_')[0]
    user_hand = HAND_DICT[username]
    if user_hand == 'right':
        arm_joints = ["Right Shoulder", "Right Elbow", "Right Wrist"]
        selected_joints_idx = [SMPL_JOINT_ORDER.index(k) for k in arm_joints]    
    else:
        arm_joints = ["Left Shoulder", "Left Elbow", "Left Wrist"]
        selected_joints_idx = [SMPL_JOINT_ORDER.index(k) for k in arm_joints]       
    
    if pred.shape !=gt.shape:
        pred = interpolate_motion_data(pred, gt.shape[0])

    c_ade, c_fde = compute_ade_fde_jointsets(pred, gt, selected_joints_idx)
        
    ade += c_ade 
    fde += c_fde 
    count += 1
    
ade = ade/count
fde = fde/count
    
print('ADE (Lim(Arm)) Joints): ', ade)
print('FDE (Lim(Arm)) Joints): ', fde)


Head

In [ ]:
import os
import numpy as np
import json

ade = 0
fde = 0
count = 0
for seq_name in pred_dict:
    
    pred_flat = pred_dict[seq_name] # nFrame x 23 x 3
    gt_flat = gt_dict[seq_name] # nFrame x 23 x 3

    T_pred = len(pred_flat) // (len(SMPL_JOINT_ORDER) * 3)
    T_gt = len(gt_flat) // (len(SMPL_JOINT_ORDER) * 3)

    pred = np.array(pred_flat).reshape((T_pred, len(SMPL_JOINT_ORDER), 3))
    gt = np.array(gt_flat).reshape((T_gt, len(SMPL_JOINT_ORDER), 3))
    
    # normalize 
    pred = pred - pred[:,:1,:]
    pred[:, :, 1], pred[:, :, 2] = pred[:, :, 2].copy(), pred[ :, :, 1].copy()
    gt = gt - gt[:,:1,:]
    
    username = seq_name.split('_')[0]
    user_hand = HAND_DICT[username]
    head_joints = ['Head']
    selected_joints_idx = [SMPL_JOINT_ORDER.index(k) for k in head_joints]        
    
    if pred.shape !=gt.shape:
        pred = interpolate_motion_data(pred, gt.shape[0])

    c_ade, c_fde = compute_ade_fde_jointsets(pred, gt, selected_joints_idx)
        
    ade += c_ade 
    fde += c_fde 
    count += 1
    
ade = ade/count
fde = fde/count
    
print('ADE (Head Joints): ', ade)
print('FDE (Head Joints): ', fde)


Mobility Aid (Cane, Guide Dog)

In [ ]:
import os
import numpy as np
import json

ade = 0
fde = 0
count = 0
for seq_name in pred_dict:
    
    pred_flat = pred_dict[seq_name] # nFrame x 23 x 3
    gt_flat = gt_dict[seq_name] # nFrame x 23 x 3

    T_pred = len(pred_flat) // (len(SMPL_JOINT_ORDER) * 3)
    T_gt = len(gt_flat) // (len(SMPL_JOINT_ORDER) * 3)

    pred = np.array(pred_flat).reshape((T_pred, len(SMPL_JOINT_ORDER), 3))
    gt = np.array(gt_flat).reshape((T_gt, len(SMPL_JOINT_ORDER), 3))
    
    # normalize 
    pred = pred - pred[:,:1,:]
    pred[:, :, 1], pred[:, :, 2] = pred[:, :, 2].copy(), pred[ :, :, 1].copy()
    gt = gt - gt[:,:1,:]
    
    username = seq_name.split('_')[0]
    user_hand = HAND_DICT[username]
    prop_joints = ['Prop']
    selected_joints_idx = [SMPL_JOINT_ORDER.index(k) for k in prop_joints]        
    
    if pred.shape !=gt.shape:
        pred = interpolate_motion_data(pred, gt.shape[0])

    c_ade, c_fde = compute_ade_fde_jointsets(pred, gt, selected_joints_idx)
        
    ade += c_ade 
    fde += c_fde 
    count += 1
    
ade = ade/count
fde = fde/count
    
print('ADE (Prop): ', ade)
print('FDE (Prop): ', fde)
